In [1]:
# importing libraries
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

import datetime

# Function-1: Prediction

In [2]:
def final_fun_1(data):
    data = data[['Dept', 'Size', 'Store', 'Type', 'IsHoliday', 'Date']]
    data['day'] = pd.DatetimeIndex(data['Date']).day
    data['Week'] = pd.DatetimeIndex(data['Date']).week
    data['year'] = pd.DatetimeIndex(data['Date']).year
    
    def isholiday_le(x):
        IsHoliday = []
        for i in x['IsHoliday']:
            if i == False:
                IsHoliday.append(0)
            else:
                IsHoliday.append(1)
        x['IsHoliday'] = IsHoliday
        return x
    data = isholiday_le(data)
        
    def type_le(x):
        store_type = []
        for i in x['Type']:
            if i == 'A':
                store_type.append(1)
            elif i == 'B':
                store_type.append(2)
            else:
                store_type.append(3)
        x['Type'] = store_type
        return x
    data = type_le(data)
    
    final_model = joblib.load('final_model.pkl')
    
    data.drop('Date', axis=1, inplace=True)
    
    predicted_weekly_sale = final_model.predict(data)
    
    return predicted_weekly_sale

In [3]:
def final_fun_2(data,actual_weekly_sale):
    data = data[['Dept', 'Size', 'Store', 'Type', 'IsHoliday', 'Date']]
    data['day'] = pd.DatetimeIndex(data['Date']).day
    data['Week'] = pd.DatetimeIndex(data['Date']).week
    data['year'] = pd.DatetimeIndex(data['Date']).year
    
    def isholiday_le(x):
        IsHoliday = []
        for i in x['IsHoliday']:
            if i == False:
                IsHoliday.append(0)
            else:
                IsHoliday.append(1)
        x['IsHoliday'] = IsHoliday
        return x
    data = isholiday_le(data)
        
    def type_le(x):
        store_type = []
        for i in x['Type']:
            if i == 'A':
                store_type.append(1)
            elif i == 'B':
                store_type.append(2)
            else:
                store_type.append(3)
        x['Type'] = store_type
        return x
    data = type_le(data)
    
    def WMAE(dataset, real, predicted):
  
        weights = []
        for i in dataset['IsHoliday']:
            if i == 1:
                weights.append(5)
            else:
                weights.append(1)
        weights = np.array(weights)
        wmae = np.sum(weights*abs(real-predicted))/(np.sum(weights))
        return wmae
    
    final_model = joblib.load('final_model.pkl')
    
    data.drop('Date', axis=1, inplace=True)
    
    predicted_weekly_sale = final_model.predict(data)
    
    wmae_score = WMAE(data, actual_weekly_sale, predicted_weekly_sale)
    
    return wmae_score

In [4]:
df = pd.read_csv('Final_data.csv')
sample_data = df.sample(1)
sample_data.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Type,Size
300805,31,2,2012-01-06,60604.24,False,48.1,3.157,6105.78,25953.41,91.69,1254.6,6256.02,219.355063,7.057,A,203750


In [5]:
predicted_weekly_sale = final_fun_1(sample_data)
wmae_score = final_fun_2(sample_data, sample_data['Weekly_Sales'])
print('Actual Weekly Sale of sampled data is: ', sample_data['Weekly_Sales'].values[0])
print('Predicted Weekly Sale of sampled data is: ', predicted_weekly_sale[0])
print('WMAE score of sampled data is: ', wmae_score)

Actual Weekly Sale of sampled data is:  60604.24
Predicted Weekly Sale of sampled data is:  58973.30691428563
WMAE score of sampled data is:  1630.933085714365
